# LEAD Data Processing

These scripts pull Low-Income Energy Affordability Data (LEAD) state zip files downloaded from: https://data.openei.org/submissions/573 and compiles a table of annual household income and energy expenditures by county and across 6 building types for households <200FPL.

The table is output into a file "LEAD_byCounty_BLD.xlsx" that is input into the main spreadsheet for energy reduction calculations.


In [7]:
#Pull in 52 state LEAD files, sum of units at all FPL levels
#only keep FIP code data

import numpy as np
import pandas as pd
import zipfile
from os import listdir, chdir
import re


#Set Research DIR
research_dir="C:/Users/helen/OneDrive/Documents/TPP/Research/"
chdir(research_dir)

#extract files from state zips downloaded from LEAD website 
def extractLeadData():
    directory_to_extract_to = "LEAD Data Extracted"
    for zipfil in listdir("LEAD Data"):
        path_to_zip_file = "LEAD Data/" + zipfil
        with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
            zip_ref.extractall(directory_to_extract_to)

#compile data for <200FPL by county and building type
#outputs to file and also returns df
def compileData():
    income_type = "FPL" #AMI, SMI, FPL
    index_cols=['FIP','LOCATION','FPL15','BLD','TEN']
    # IMPT: because this takes unweighted mean, must use FUEL*UNITS, also want to use sum, not mean, for units
    val_cols = {'UNITS':np.sum,'ELEP*UNITS':np.sum,'GASP*UNITS':np.sum,'FULP*UNITS':np.sum,'HINCP*UNITS':np.sum}
    
    chdir("LEAD Data Extracted")
    i=0
    
    #loop through files
    for fil in listdir():
        
        # search for file names with the following format: ["AL FPL State, Counties, Cities 2018.csv"]
        relevant_file_names = ".. "+income_type+" State, Counties, Cities 2018.csv"
        if re.search(relevant_file_names, fil):
            print(f"Adding file: {fil}")
            if i==0: #initialize dataframe
                df=pd.read_csv(fil)
                #If dict is passed, the key is column to aggregate and value is function or list of functions.
                df=df.pivot_table(index=index_cols,aggfunc=val_cols) 
                
            else: #append to dataframe
                nextstate=pd.read_csv(fil).pivot_table(index=index_cols,aggfunc=val_cols) 
                df=pd.concat([df,nextstate])
            i+=1
            
    df =  df.rename_axis(None, axis=1).reset_index() 
    #only keep rows <200FPL and take avg over separate FPL categories
    df=df[df['FPL15'].isin(["0-100%","100-150%","150-200%"])]
    index_cols2=['FIP','LOCATION','BLD']
    df=df.pivot_table(index=index_cols2,aggfunc=val_cols)
    df =  df.rename_axis(None, axis=1).reset_index() 
    
    #Only keep rows with 5 digit FIPS, and those with leading 0s cut off
    df = df[df['FIP']<=99999]
    df = df[df['FIP']>=999]
    
    df['FIP']=df['FIP'].astype(str)
    df['FIP']=df['FIP'].str.zfill(5)
    
    df.to_csv(research_dir+"LEAD_full file.xlsx")
    print("Complete")
    
    return df  
    

In [8]:
df_nochange=compileData()
df_nochange.head()

Adding file: AK FPL State, Counties, Cities 2018.csv
Adding file: AL FPL State, Counties, Cities 2018.csv
Adding file: AR FPL State, Counties, Cities 2018.csv
Adding file: AZ FPL State, Counties, Cities 2018.csv
Adding file: CA FPL State, Counties, Cities 2018.csv
Adding file: CO FPL State, Counties, Cities 2018.csv
Adding file: CT FPL State, Counties, Cities 2018.csv
Adding file: DC FPL State, Counties, Cities 2018.csv
Adding file: DE FPL State, Counties, Cities 2018.csv
Adding file: FL FPL State, Counties, Cities 2018.csv
Adding file: GA FPL State, Counties, Cities 2018.csv
Adding file: HI FPL State, Counties, Cities 2018.csv
Adding file: IA FPL State, Counties, Cities 2018.csv
Adding file: ID FPL State, Counties, Cities 2018.csv
Adding file: IL FPL State, Counties, Cities 2018.csv
Adding file: IN FPL State, Counties, Cities 2018.csv
Adding file: KS FPL State, Counties, Cities 2018.csv
Adding file: KY FPL State, Counties, Cities 2018.csv
Adding file: LA FPL State, Counties, Cities 20

,FIP,LOCATION,BLD,ELEP*UNITS,FULP*UNITS,GASP*UNITS,HINCP*UNITS,UNITS
520,01001,"Autauga County, AL",1 ATTACHED,3.046679e+04,9.369043e-09,1.434492e+01,5.703275e+05,36.723065
521,01001,"Autauga County, AL",1 DETACHED,6.690664e+06,7.417265e+04,2.481871e+06,8.246946e+07,3920.426148
522,01001,"Autauga County, AL",10-19 UNIT,1.295270e+05,4.928960e-08,1.019095e-06,1.644347e+06,105.271472
523,01001,"Autauga County, AL",2 UNIT,6.558260e+04,4.796632e-08,4.344278e+04,1.699168e+06,131.308750
524,01001,"Autauga County, AL",20-49 UNIT,6.715486e+04,3.252238e-08,6.638352e-07,1.420555e+06,82.374785


In [10]:
#Extra processing for labeling building types and reindexing for incorporation into the main file

df=df_nochange

#Get State ID from FIPS code for matching
df['State ID'] = df['FIP'].str[:2]

#want dataframe with fuels by BLD combined categories,by cty
#create new column for each housing category
df['Housing Unit #'] = 0 

df.loc[df['BLD']=='1 DETACHED', 'Housing Unit #'] = 1
df.loc[df['BLD']=='1 ATTACHED', 'Housing Unit #'] = 2
df.loc[df['BLD'].isin(['2 UNIT','3-4 UNIT']), 'Housing Unit #'] = 3
df.loc[df['BLD'].isin(['5-9 UNIT','10-19 UNIT','20-49 UNIT','50+ UNIT']), 'Housing Unit #'] = 4
df.loc[df['BLD']=='MOBILE_TRAILER', 'Housing Unit #'] = 5

#remove 0s ('BOAT_RV_VAN')
df=df[df['Housing Unit #']>0]
# print(np.unique(df['BLD']))

#pivotsum on Housing Unit
index_cols3=['FIP','LOCATION','State ID','Housing Unit #']
val_cols = {'UNITS':np.sum,'ELEP*UNITS':np.sum,'GASP*UNITS':np.sum,'FULP*UNITS':np.sum,'HINCP*UNITS':np.sum}

df=df.pivot_table(index=index_cols3,aggfunc=val_cols)
df =  df.rename_axis(None, axis=1).reset_index() 

#output 
df.to_excel(research_dir+"LEAD_byCounty_BLD.xlsx")
